05-19-2020: Por enquanto, as tentativas de unir dados das bases de dados esta sendo difícil já que boa parte das informações de celulares não consegue ser encaixada com os seus respectivos preços. Por isso, optou-se por usar dados do site Jacotei.com,
já que ele forneceria um MVP (Mínimo Produto Viável - É uma espécie de produto simplificado e sem implementação de todas as funcionalidade para teste de mercado/hipóteses de funcionalidade/etc) e com isso um protótipo do pipeline do modelo de ML

In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd
import re
from datetime import datetime
import time
from selenium import webdriver
import math

# Mini-Pipeline forJacotei.com

## Data extraction

Juntar as tabelas se mostrou infrutifero, a ideia agora é usar o jacotei...

In [2]:
def get_soup_from_url(url):
    """This function gets an url and returns a bs4 soup"""
    
    headers = {
            'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36',
        }
    response = requests.get(url,headers=headers)
    soup = BeautifulSoup(response.content)
    return soup

In [3]:
def plain_columns(dataframe):
    """This function removes accentuation from columns names and lowers letters.
    Receives a dataframe and returns the treated DataFrame"""

    
    from unidecode import unidecode
    
    unidecode_dict = {}
    for col in dataframe.columns.tolist():
        unidecode_dict[col] = unidecode(col).lower()
    dataframe.rename(columns=unidecode_dict, inplace=True)        
    
    return dataframe

In [4]:
jacotei = pd.read_csv('../pipeline/data_storage/temp/toDB2020-05-19.csv', encoding='cp1252')

In [5]:
def extract_data_from_each_smartphone_link(main_dataframe, link_column_name, backup_path):
    """This function receives a dataframe result from the extractions of the main pages of smartphones from
    maiscelular.com and extract information from each smartphone and returns it as a dataframe.
    It also saves a dataframe in a backup. It there is no backup, it starts from the beginning,
    however, in the contrary case, it loads the backup and restarts from where it stopped."""
    
    # load backup
    try:
        backup = pd.read_csv(backup_path, encoding='utf-8')
    except:
        columns = ['link']
        backup = pd.DataFrame(columns = columns)
    
    new_results = backup.copy()
    
    # check which links will be searched
    links_already_searched = backup.link.unique()
    mask = main_dataframe.loc[:,link_column_name].apply(lambda x: x not in links_already_searched)
    links_not_searched = main_dataframe.loc[mask].loc[:,link_column_name]
    
    if len(links_not_searched) == 0:
        print('There are no new smartphones to extract!')
        return backup 
    
    print(f'Extracting information from {len(links_not_searched)} new smartphones...')
    time.sleep(1)
    
    for link in tqdm(links_not_searched):
        
        try:
            #extract
            df_html = pd.read_html(link,encoding='utf-8')[0]
            #treat
            df_html = df_html.set_index(df_html.columns[0]).T
            df_html = plain_columns(df_html)
            df_html['html_error'] = False
        except:
            df_html = pd.DataFrame({'html_error': True}, index=[0])
        
        #add link
        df_html['link'] = link
        
        # add to the final result and backup
        new_results = pd.concat([new_results, df_html], ignore_index=True).reset_index(drop=True)
        new_results.to_csv(backup_path, encoding='utf-8', index=False)
    
    return new_results

In [6]:
extracted = extract_data_from_each_smartphone_link(main_dataframe=jacotei,
                                                    link_column_name = 'link_anuncio',
                                                    backup_path='jacotei_backup.csv')

Extracting information from 5 new smartphones...


100%|██████████| 5/5 [00:00<00:00,  5.15it/s]


## Merging data with original DataFrame

In [16]:
data = pd.merge(left=jacotei, right=extracted,left_on='link_anuncio', right_on='link')

## Treating data for Pipeline

In [20]:
#unused columns
data.drop(columns=['maior_preco','destino_do_link', 'link_anuncio', 'timestamp', 'link', 'smartphone'])

,modelo_celular,menor_preco,data_anuncio,tela,touchscreen,4g,3g,camera traseira,camera selfie,nfc,...,peso produto,cest,unidade de medida,cor principal,quantidade de imeis,lastro,camada,certificado anatel,dimensoes do produto,bateria
0,Smartphone Samsung Galaxy S10 SM-G973F Desbloq...,"R$ 40.999,00",Mai/2020,"6.1""",Sim,Sim,Sim,16.0 Megapixels,10 Megapixels,Sim,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Smartphone Huawei P20 Pro Desbloqueado 128GB A...,"R$ 16.935,12",Jul/2018,"6.1""",Sim,Sim,Sim,40 Megapixels,24 Megapixels,Sim,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Recuperado iPhone 8 64GB / 256GB Desbloqueado-...,"R$ 14.527,39",Abr/2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,iPhone 11 Pro Max 512GB Prateado iOS 4G + Wi-F...,"R$ 9.599,00",Fev/2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Smartphone Motorola Razr 128GB Dual Chip Andro...,"R$ 8.999,00",Abr/2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2360,Celular Huawei U6020 3G Wi fi Preto,"R$ 79,00",Nov/2017,"2.4""",Não,Não,Sim,NaN,1.3 Megapixels,Não,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2361,Celular Infinity W201 Desbloqueado Dual Chip P...,"R$ 75,91",Jul/2019,1.7'',Não,Não,Não,0.8 Megapixels,0.3 Megapixels,Não,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2362,Celular Lenoxx CX 904 Desbloqueado Dual Chip P...,"R$ 74,00",Jun/2018,"1.8""",Não,Não,Não,NaN,NaN,Não,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2363,Celular Para Idosos DTC Myphone M1 Desbloquead...,"R$ 69,19",Nov/2018,"1.3""",Não,Não,Não,NaN,NaN,Não,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
data.columns

Index(['modelo_celular', 'maior_preco', 'menor_preco', 'data_anuncio',
       'destino_do_link', 'link_anuncio', 'timestamp', 'link', 'smartphone',
       'tela', 'touchscreen', '4g', '3g', 'camera traseira', 'camera selfie',
       'nfc', 'gravacao de video', 'conexoes', 'funcoes', 'dual chip',
       'altura', 'largura', 'profundidade', 'formato', 'flash embutido',
       'memoria interna', 'modelo', 'operadora', 'peso', 'sistema operacional',
       'sugestoes', 'html_error', 'tecnologia', 'recursos da tela',
       'bateria em conversacao', 'processador', 'tamanho da tela',
       'resolucao da tela', 'tecnologia da tela', 'slot de expansao', 'marca',
       '1x', '2x', '3x', '4x', '5x', '6x', '7x', '8x', '9x', '10x',
       'total de r$ 629,00 parcelado sem juros', 'nome', 'garantia (meses)',
       'peso produto', 'cest', 'unidade de medida', 'cor principal',
       'quantidade de imeis', 'lastro', 'camada', 'certificado anatel',
       'dimensoes do produto', 'bateria'],
      d